In [1]:
#Import PyTorch and matplotlib
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import time
from pthflops import count_ops

vRead = iio.imread("c_elegans.mp4")
video = np.array(vRead)

#Check PyTorch version
torch.__version__


'1.13.1'

In [2]:
print("Cuda available: ", torch.cuda.is_available())
if(torch.cuda.is_available()):
    torch.cuda.set_device("cuda:1")

    print("Is cuDNN version:", torch.backends.cudnn.version())

    print("cuDNN enabled:a", torch.backends.cudnn.enabled)

    print("Device count: ", torch.cuda.device_count())

    print("Current device: ", torch.cuda.current_device())

    print("Device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))
#Setup device agnostic code (i.e use GPU if possible)
device = "cuda" if torch.cuda.is_available() else "cpu"
gpuNum = 1
print(device)

Cuda available:  True
Is cuDNN version: 8302
cuDNN enabled:  True
Device count:  4
Current device:  1
Device name:  NVIDIA TITAN Xp
cuda


In [3]:
#Video Metadata
import imageio.v3 as iio
props = iio.improps("c_elegans.mp4")
print("Shape (frames, w, h, RGB): \n" + str(props.shape))
numFrames = props.shape[0]
print(props.dtype)

Shape (frames, w, h, RGB): 
(2484, 322, 344, 3)
uint8


### Encoding Image as a Tensor

In [4]:
#Input grid  (width_px, height_ px)
grid = torch.empty(400, props.shape[1], props.shape[2], 3).to(device)

print(numFrames)
# Create image tensor
videoIn = torch.as_tensor(video[:400])
videoIn.shape




2484


torch.Size([400, 322, 344, 3])

### Dataset and DataLoader

In [18]:
import os
from torch.utils.data import DataLoader

#Max number of frames ---> numFrames   variable 

class SingleImageDataset(Dataset):
    def __init__(self, videoIn, transform=None, target_transform=None):
        self.videoIn = videoIn.type(torch.int32)
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return int(image.shape[0]) * int(image.shape[1])
    def __getitem__(self, idx):
        row = idx // int(image.shape[1])
        col = idx % int(image.shape[1])
        #Generate random frame
        frameLimit = videoIn.shape[0]
        currFrame = torch.randint(0, frameLimit, (1,)).item()
        pixel = torch.as_tensor(self.videoIn[currFrame][row][col]).type(torch.int32).cpu()
        #label = pixel
        return torch.as_tensor([currFrame, row, col]).cpu(), pixel
training_data = SingleImageDataset(videoIn)
#testing_data = None

#train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
#train_dataloader = DataLoader(testing_data, batch_size=32, shuffle=True)


In [17]:
currFrame = torch.randint(0, 400, (1,)).item()
currFrame

109

### Decoding video data from MLP

In [19]:
def writeVideoMP4(videoTensor, filename='decoded'):
    w = iio.get_writer(filename + '.mp4', format='FFMPEG', mode='I', fps=20,
                        codec='h264_vaapi',
                        output_params=['-vaapi_device',
                                        '/dev/dri/renderD128',
                                        '-vf',
                                        'format=gray|nv12,hwupload'],
                        pixelformat='vaapi_vld')
    for frame in videoTensor:
        w.append_data(frame)
    w.close()
    print('video saved in local directory as: ' + filename + '.mp4')
    return None

In [20]:
def writeVideoGIF(videoTensor, filename='decoded'):
    fps = 30
    dur = 1000 * (1/fps)
    iio.imwrite(filename + '.gif', videoTensor.cpu().numpy(), duration=dur)
    print('video saved in local directory as: ' + filename + '.gif')
    return None

In [ ]:
#FOR TESTING THE VIDEO WRITER
writeVideoGIF(videoIn, 'example')

### MLP-based Model

In [23]:
import torch
from torch import nn
#from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms

#Use seed for reproducibility
torch.manual_seed(42)
#Multilayer Percepetron Model 
class MLP(nn.Module):
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
        nn.Linear(input_shape, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, output_shape),
        )
    #forward reconstruction
    def forward(self, X):
        return self.layer_stack(X.to(device))

### Instance

In [24]:
#Create an Instance and set loss function & optimizer
model_0 = MLP(input_shape=3, 
              hidden_units=128, 
              output_shape=3).to(device)


In [34]:

lr1 = 0.00001
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=lr1)
#list(model_0.parameters())

### Reconstruction Loop:  NN --> Tensor --> GIF

In [38]:
# Tests to reconstruct image for some frame
def test_loop():
    model_0.eval()
    testVideo = torch.zeros(videoIn.shape[0], videoIn.shape[1], videoIn.shape[2], 3).type(torch.int32)
    with torch.inference_mode():
        for i in range(0,10):
            for j in range(0, testVideo.shape[1]):
                for k in range(0, testVideo.shape[2]):
                    testVideo[i][j][k] = model_0(torch.tensor([1.0 * i, 1.0 * j, 1.0 * k])).type(torch.float32)
    writeVideoGIF(testVideo, 'reconstruction')
    return None

### Training Loop

In [32]:
from tqdm import tqdm

train_dataloader = DataLoader(videoIn, batch_size=128, shuffle=True, pin_memory=True)

def trainLoop():
  #Track values
  #Loop through data
  for epoch in tqdm(range(0, 100)):
      ### Training
      model_0.train()
      #Process DataLoader Batch
      for batch in iter(train_dataloader):
        #print("new batch!")
        coords = torch.as_tensor(batch[0]).to(device)
        pixels = torch.as_tensor(batch[1]).to(device)
        #Loop through each pixel in batch
        
        X = torch.as_tensor(coords).type(torch.float32).to(device)
        y_train = torch.as_tensor(pixels).type(torch.float32).to(device)
        
        #1. Forward Pass
        y_pred = model_0(X).to(device)
        #2. Calculate Loss
        loss = loss_fn(y_pred, y_train)
        #3. Optimizer Zero Grad
        optimizer.zero_grad()
        #4. Backpropagation
        loss.backward()
        #5. Step optimizer
        optimizer.step()
        
      if epoch % 2 == 0:
        print(f"Epoch: {epoch} | LR: {lr1} | Train loss: {loss}")
        #test_loop()

  print('Training finished')

In [33]:
trainLoop()

  1%|          | 1/100 [00:00<01:19,  1.25it/s]

Epoch: 0 | LR: 0.0001 | Train loss: 908.991455078125


  3%|▎         | 3/100 [00:01<00:43,  2.23it/s]

Epoch: 2 | LR: 0.0001 | Train loss: 1131.4500732421875


  5%|▌         | 5/100 [00:02<00:48,  1.94it/s]

...

Epoch: 96 | LR: 0.0001 | Train loss: 819.2588500976562


 99%|█████████▉| 99/100 [00:43<00:00,  2.13it/s]

Epoch: 98 | LR: 0.0001 | Train loss: 781.194580078125


100%|██████████| 100/100 [00:43<00:00,  2.30it/s]

Training finished


In [39]:
test_loop()

### Save Model

In [ ]:
### Save Model State Dictionary
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "video_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
MODEL_SAVE_PATH

#3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)
print(MODEL_SAVE_PATH)

### Loading a Model

In [ ]:
# To load in a saved state_dict we have to instantiate a new instance of our model class
#loaded_model_0 = MLP() # new instance!

#Load the saved state_dict of model_0 (this wil updated the new instance with the updated parameters.)
model_0.load_state_dict(torch.load(f='models/video_model_0.pth'))

### Calculate Raw Model Size

In [ ]:
# Check model size by summing parameters and state_dict
params_size = 0
for param in model_0.parameters():
    params_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model_0.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (params_size + buffer_size) / 1024**2
videoSize = videoIn.shape[0] * (8 * videoIn.shape[1] * videoIn.shape[2] * 3)
videoSizeMB = videoSize / (10**6)
perDecrease = (videoSizeMB - size_all_mb) / videoSizeMB
perDecrease *= 100
print('original image size(no compression): {:.3f}MB'.format(videoSizeMB))
print('model size: {:.3f}MB'.format(size_all_mb))
print('Percent decrease in memory size: {:.3f}%'.format(perDecrease))

In [ ]:
### Kill Kernel
quit()